In [1]:
import circuit
import gate 
import draw
import state
import numpy as np
import torch 
from scipy import special
import sys 
import photonic_qmath

In [2]:
import perceval as pcvl
import perceval.components as comp 
from perceval . components import BS
import numpy as np

In [3]:
import time 

In [5]:
## for state list
for test_id in range(100):
    nmode = np.random.randint(2,7)
    ndevice = np.random.randint(10,100)
    ini_state_pre = np.random.randn(nmode)
    ini_state_pre = ini_state_pre-ini_state_pre.min()
    ini_state_pre = np.round(ini_state_pre/ini_state_pre.sum()*nmode)
    ini_state_pre[ini_state_pre<0] = 0
    if ini_state_pre.sum()!=nmode:
        ini_state_pre[ini_state_pre.argmax()] += nmode - ini_state_pre.sum()
    ini_state = [int(i) for i in ini_state_pre]
    ini_state_ts = [(1, ini_state)]
    assert np.sum(ini_state_pre)==nmode
    
    test_gate = ( pcvl . Circuit (nmode, name =" test1 "))

    dq_gate = circuit.QumodeCircuit(nmode=nmode, init_state=ini_state, name="test", cutoff = sum(ini_state)+1, basis = True)

    encode = True

    for i in range(ndevice): ## take the random circuit
        j = np.random.uniform(-1,1)
        if j>0:
            temp_1 = int(np.random.choice(np.arange(nmode)))
            angle_1 = np.random.uniform(0,2*np.pi)
            test_gate.add((temp_1),comp.PS(angle_1))
            dq_gate.ps(angle_1, [temp_1],encode=encode)
        else:
            k = int(np.random.choice(np.arange(nmode-1)))
            angle_2 = np.random.uniform(0,2*np.pi)
            test_gate.add((k,k+1), BS.Rx(angle_2))
            dq_gate.bs(angle_2/2, [k, k+1],encode=encode)
            
    
    t1 = time.perf_counter()
    backend = pcvl.BackendFactory().get_backend("Naive")
    backend.set_circuit(test_gate)
    input_state=pcvl.BasicState(ini_state)
    backend.set_input_state(input_state)
    re1 = backend.evolve()
    t2 = time.perf_counter()
    T1 = t2-t1

    t3 = time.perf_counter()
    re2 = dq_gate()
    t4 = time.perf_counter()
    T2 = t4-t3 

    ## calculating the difference for two simu approach
    max_error = -1.0
    for key in re1.keys():
        key2 = list(key)
        key3 = state.FockState(nmode, key2)
        tmp_error =  abs(re2[(key3)]-re1[(key)])
#         tmp_error =  abs(re2[tuple(key2)]-re1[(key)])
        if tmp_error > max_error:
            max_error = tmp_error
    if max_error>1e-3:
        raise Exception("check this case")
    print(f"test_id: {test_id} | max_difference: {'%.4e' %max_error} | T1: {'%.2f' %T1}s | T2: {'%.2f' %T2}s | nm: {nmode} | nd: {ndevice}")

test_id: 0 | max_difference: 2.7476e-07 | T1: 0.00s | T2: 0.22s | nm: 3 | nd: 57
test_id: 1 | max_difference: 3.9448e-06 | T1: 0.05s | T2: 4.85s | nm: 6 | nd: 99
test_id: 2 | max_difference: 2.3842e-07 | T1: 0.00s | T2: 0.05s | nm: 2 | nd: 35
test_id: 3 | max_difference: 2.5832e-07 | T1: 0.00s | T2: 0.15s | nm: 3 | nd: 50
test_id: 4 | max_difference: 1.2926e-06 | T1: 0.06s | T2: 4.88s | nm: 6 | nd: 96
test_id: 5 | max_difference: 2.1044e-06 | T1: 0.02s | T2: 1.42s | nm: 5 | nd: 99
test_id: 6 | max_difference: 4.5204e-07 | T1: 0.00s | T2: 0.15s | nm: 3 | nd: 29
test_id: 7 | max_difference: 2.4074e-07 | T1: 0.00s | T2: 0.15s | nm: 3 | nd: 34
test_id: 8 | max_difference: 4.2936e-07 | T1: 0.05s | T2: 4.86s | nm: 6 | nd: 34
test_id: 9 | max_difference: 5.4593e-07 | T1: 0.05s | T2: 5.00s | nm: 6 | nd: 49
test_id: 10 | max_difference: 5.1382e-07 | T1: 0.00s | T2: 0.15s | nm: 3 | nd: 57
test_id: 11 | max_difference: 8.8337e-07 | T1: 0.05s | T2: 4.85s | nm: 6 | nd: 35
test_id: 12 | max_differen

KeyboardInterrupt: 

In [18]:
## for state tensor
for test_id in range(100):
    nmode = np.random.randint(2,7)
    ndevice = np.random.randint(10,100)
    ini_state_pre = np.random.randn(nmode)
    ini_state_pre = ini_state_pre-ini_state_pre.min()
    ini_state_pre = np.round(ini_state_pre/ini_state_pre.sum()*nmode)
    ini_state_pre[ini_state_pre<0] = 0
    if ini_state_pre.sum()!=nmode:
        ini_state_pre[ini_state_pre.argmax()] += nmode - ini_state_pre.sum()
    ini_state = [int(i) for i in ini_state_pre]
    ini_state_ts = [(1, ini_state)]
    assert np.sum(ini_state_pre)==nmode
    
    test_gate = ( pcvl . Circuit (nmode, name =" test1 "))

    dq_gate = circuit.QumodeCircuit(nmode=nmode, init_state=ini_state_ts, name="test", cutoff = sum(ini_state)+1, basis = False)

    encode = True

    for i in range(ndevice): ## take the random circuit
        j = np.random.uniform(-1,1)
        if j>0:
            temp_1 = int(np.random.choice(np.arange(nmode)))
            angle_1 = np.random.uniform(0,2*np.pi)
            test_gate.add((temp_1),comp.PS(angle_1))
            dq_gate.ps(angle_1, [temp_1],encode=encode)
        else:
            k = int(np.random.choice(np.arange(nmode-1)))
            angle_2 = np.random.uniform(0,2*np.pi)
            test_gate.add((k,k+1), BS.Rx(angle_2))
            dq_gate.bs(angle_2/2, [k, k+1],encode=encode)
            
    
    t1 = time.perf_counter()
    backend = pcvl.BackendFactory().get_backend("Naive")
    backend.set_circuit(test_gate)
    input_state=pcvl.BasicState(ini_state)
    backend.set_input_state(input_state)
    re1 = backend.evolve()
    t2 = time.perf_counter()
    T1 = t2-t1

    t3 = time.perf_counter()
    re2 = dq_gate()
    t4 = time.perf_counter()
    T2 = t4-t3 

    ## calculating the difference for two simu approach
    max_error = -1.0
    for key in re1.keys():
        key2 = list(key)
#         key3 = state.FockState(nmode, key2)
#         tmp_error =  abs(re2[(key3)]-re1[(key)])
        tmp_error =  abs(re2[tuple(key2)]-re1[(key)])
        if tmp_error > max_error:
            max_error = tmp_error
    if max_error>1e-3:
        raise Exception("check this case")
    print(f"test_id: {test_id} | max_difference: {'%.4e' %max_error} | T1: {'%.2f' %T1}s | T2: {'%.2f' %T2}s | nm: {nmode} | nd: {ndevice}")

test_id: 0 | max_difference: 5.9381e-07 | T1: 0.00s | T2: 0.24s | nm: 3 | nd: 90
test_id: 1 | max_difference: 8.7207e-07 | T1: 0.01s | T2: 0.29s | nm: 3 | nd: 99
test_id: 2 | max_difference: 4.1723e-07 | T1: 0.00s | T2: 0.04s | nm: 2 | nd: 32
test_id: 3 | max_difference: 9.9202e-07 | T1: 0.00s | T2: 0.08s | nm: 2 | nd: 65
test_id: 4 | max_difference: 1.2700e-06 | T1: 0.00s | T2: 0.13s | nm: 2 | nd: 97
test_id: 5 | max_difference: 3.0224e-07 | T1: 0.01s | T2: 0.28s | nm: 4 | nd: 72
test_id: 6 | max_difference: 3.8831e-07 | T1: 0.01s | T2: 0.41s | nm: 4 | nd: 92
test_id: 7 | max_difference: 2.9143e-07 | T1: 0.01s | T2: 0.67s | nm: 5 | nd: 67
test_id: 8 | max_difference: 1.6660e-07 | T1: 0.05s | T2: 0.12s | nm: 6 | nd: 11
test_id: 9 | max_difference: 1.6408e-07 | T1: 0.00s | T2: 0.08s | nm: 4 | nd: 24
test_id: 10 | max_difference: 8.3778e-07 | T1: 0.00s | T2: 0.10s | nm: 2 | nd: 79
test_id: 11 | max_difference: 4.6063e-07 | T1: 0.01s | T2: 0.34s | nm: 4 | nd: 67
test_id: 12 | max_differen

KeyboardInterrupt: 

## test 6mode

In [11]:
nmode = 6 
ndevice = 100

ini_state = [1, 1, 1,1,1,1]
test_gate = ( pcvl . Circuit (nmode, name =" test1 "))

dq_gate = circuit.QumodeCircuit(nmode=nmode, init_state=ini_state, name="test", cutoff = sum(ini_state)+1, basis = True) # basis=True, using state list

for i in range(ndevice): ## take the random circuit
    j = np.random.uniform(-1,1)
    if j>0:
        temp_1 = int(np.random.choice(np.arange(nmode)))
        angle_1 = np.random.uniform(0,2*np.pi)
        test_gate.add((temp_1),comp.PS(angle_1))
        dq_gate.ps(angle_1, [temp_1])
    else:
        k = int(np.random.choice(np.arange(nmode-1)))
        angle_2 = np.random.uniform(0,2*np.pi)
        test_gate.add((k,k+1), BS.Rx(angle_2))
        dq_gate.bs(angle_2/2, [k, k+1])

In [12]:
dq_gate.draw()

In [13]:
ini_state = ini_state
backend = pcvl.BackendFactory().get_backend("Naive")
backend.set_circuit(test_gate)
input_state=pcvl.BasicState(ini_state)
backend.set_input_state(input_state)
re1 = backend.evolve()
re1

StateVector(float,
            {|5,1,0,0,0,0>: (0.006864455546974607-0.0020317447075387887j),
             |5,0,1,0,0,0>: (0.00010341205223094806+0.007125456167568667j),
             |5,0,0,1,0,0>: (-0.0033748169411218793-0.005638304961251407j),
             |5,0,0,0,1,0>: (0.0034355048226369445-0.009529794183405535j),
             |5,0,0,0,0,1>: (0.004631754281311036+0.008525077027922025j),
             |4,2,0,0,0,0>: (0.0023890983214327776+0.008321039841040061j),
             |4,1,1,0,0,0>: (-0.025873569981730758+0.006046995654871445j),
             |4,1,0,1,0,0>: (-0.002792173542902024-0.002851096773261769j),
             |4,1,0,0,1,0>: (0.011431215946623796-0.020808556849772616j),
             |4,1,0,0,0,1>: (-0.01396915085049003-0.028205613638397388j),
             |4,0,2,0,0,0>: (-0.000629973346972603-0.02909519568845411j),
             |4,0,1,1,0,0>: (0.011917463409520209+0.008892840057272616j),
             |4,0,1,0,1,0>: (0.012212568904582277+0.03715538151991885j),
           

In [14]:
re2 = dq_gate()
re2 

{|300030>: tensor([-0.0590+0.0939j]),
 |021201>: tensor([0.1042-0.0359j]),
 |210201>: tensor([-0.0748+0.0744j]),
 |300012>: tensor([-0.0826-0.0616j]),
 |210120>: tensor([-0.0982-0.0219j]),
 |040101>: tensor([-0.0698+0.0701j]),
 |021030>: tensor([-0.0138-0.0972j]),
 |200301>: tensor([-0.0463+0.0852j]),
 |301002>: tensor([-0.0954-0.0107j]),
 |030012>: tensor([0.0278+0.0910j]),
 |220101>: tensor([-0.0779+0.0525j]),
 |200103>: tensor([-0.0590-0.0726j]),
 |111012>: tensor([0.0386+0.0849j]),
 |201003>: tensor([-0.0691-0.0619j]),
 |002022>: tensor([-0.0174+0.0878j]),
 |030003>: tensor([-0.0665-0.0598j]),
 |102201>: tensor([0.0765+0.0411j]),
 |004101>: tensor([0.0017-0.0866j]),
 |210003>: tensor([-0.0670-0.0544j]),
 |050001>: tensor([-0.0602-0.0614j]),
 |103101>: tensor([-0.0856+0.0066j]),
 |200400>: tensor([0.0855-0.0004j]),
 |112101>: tensor([-0.0813-0.0218j]),
 |211002>: tensor([-0.0261-0.0794j]),
 |010050>: tensor([-0.0820+0.0159j]),
 |104010>: tensor([0.0811+0.0186j]),
 |201210>: tensor([